## Using Aprioi to analyse the olympics history


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder


/home/nimisha/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nimisha/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv("dataSource/athlete_events.csv")
#data.info()
print(data.shape)
data.head(1)

(271116, 15)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN


# Data preprocessing

In [3]:
print(data.shape)
print(data.columns)
print(data.isna().sum())

(271116, 15)
Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')
ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64


## The data has 9474 null values in Age , hence will be substituting with the mode

In [4]:
AgeMode = int(data['Age'].mode()[0])
print('Age Mode', AgeMode)
data['Age'] = data['Age'].fillna(AgeMode)
data['Age'].isna().sum()

Age Mode 23


0

## 22% of the Height and 23% of the Weight column is having NaN values
## The NaN values are substitued with mode

In [5]:
HeightMode = data['Height'].mode()[0] ## 0 -> the column wise mode 
print('Height Mode',HeightMode)
data['Height'] = data['Height'].fillna(HeightMode)
data['Height'].isna().sum()

Height Mode 180.0


0

In [6]:
WeightMode = data['Weight'].mode()[0]
print('Weight Mode',WeightMode)
data['Weight'] = data['Weight'].fillna(WeightMode)
data['Weight'].isna().sum()

Weight Mode 70.0


0

## The Medal column has NaN values if no medal is won , we Substitute the NaN values with a string 'None'

In [7]:
data['Medal'] = data['Medal'].fillna('None')
data['Medal'].isna().sum()

0

## After Removing the NaN values

In [8]:
data.isna().sum()

ID        0
Name      0
Sex       0
Age       0
Height    0
Weight    0
Team      0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
dtype: int64

## We want to find the frequent itemsets for the following columns
###    ['Sex','Age', 'Height','Weight','Team','Medal']

In [9]:

# TO group a range of values together
def convertDataToRange(data,col,span):
    column = []
    for index,rows in data.iterrows():
        i = data.loc[index,col]
        temp = int(i/span)*span
        val = '(',col,':',str(temp),'-',str(temp+span),')'
        
        column.append("".join(val))
        #print(i,temp,val)
    return column       


In [10]:
def makeSelectiveSportDataset(fromData,sport,colList):
    toData = fromData[fromData['Sport'] == sport]
    toData =toData[colList]
    #to.head(5)
    med = ['Gold','Silver','Bronze']
    toData = toData[toData.Medal.isin(med)]
    return(toData)


In [11]:
def convertDataForApriori(data):
    data['Height']  = convertDataToRange(data,'Height',10)

    data['Weight']  = convertDataToRange(data,'Weight',10)

    data['Age']  = convertDataToRange(data,'Age',10)
    return data

In [12]:
def applyApriori(dataset,support):
    rec = dataset.values.tolist()
    # Finding Frequent Item Sets

    te = TransactionEncoder()
    te_ary = te.fit(rec).transform(rec)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Applying Aprioi Algo
    from mlxtend.frequent_patterns import apriori
    
    freq_Itemsets =  apriori(df,min_support = support, use_colnames = True)
    freq_Itemsets['length'] = freq_Itemsets['itemsets'].apply(lambda x:len(x))
    return(freq_Itemsets[freq_Itemsets['length'] > 1])

# Finding Frequent ItemSets on different Sports

# Atheletics

In [13]:
Athletics = makeSelectiveSportDataset(data,'Athletics',['Sex','Age', 'Height','Weight','Team','Medal'])

Athletics= convertDataForApriori(Athletics)

freq_Itemsets_Athletics = applyApriori(Athletics,0.4)
print("Athletics Data")
print(freq_Itemsets_Athletics)

Athletics Data
   support          itemsets  length
2  0.56639  ((Age:20-30), M)       2


### Result:
     1. Age group between 20-30 is common.
     2. Height and Weight does not matter.
     3. Men have a higher chance of winning

# Archery

In [14]:
Archery = makeSelectiveSportDataset(data,'Archery',['Sex','Age', 'Height','Weight','Team','Medal'])
Archery= convertDataForApriori(Archery)
freq_Itemsets_Archery = applyApriori(Archery,0.4)
print("Archery Data")

print(freq_Itemsets_Archery)

Archery Data
    support                               itemsets  length
4  0.470255     ((Height:180-190), (Weight:70-80))       2
5  0.509915                  ((Height:180-190), M)       2
6  0.487252                    ((Weight:70-80), M)       2
7  0.413598  ((Height:180-190), (Weight:70-80), M)       3


### Result:
     1. Age distribution is even.
     2. Height and Weight ((Height:180-190), (Weight:70-80)) is found to be common .


# Basket Ball

In [15]:
BasketBall = makeSelectiveSportDataset(data,'Basketball',['Sex','Age', 'Height','Weight','Team','Medal'])

BasketBall= convertDataForApriori(BasketBall)

freq_Itemsets_BasketBall =  applyApriori(BasketBall,0.4)
print("Basket Ball Data")

freq_Itemsets_BasketBall

Basket Ball Data


,support,itemsets,length
2,0.535185,"((Age:20-30), M)",2


### Result:
     1. Age group between 20-30 is common.
     2. Height and Weight does not matter.


# Boxing

In [16]:
Boxing = makeSelectiveSportDataset(data,'Boxing',['Sex','Age', 'Height','Weight','Team','Medal'])
Boxing= convertDataForApriori(Boxing)

freq_Itemsets_Boxing = applyApriori(Boxing,0.4)
print("Boxing Data")
freq_Itemsets_Boxing

Boxing Data


,support,itemsets,length
4,0.811441,"((Age:20-30), M)",2
5,0.408898,"((Height:180-190), M)",2
6,0.454449,"(Bronze, M)",2


### Result:
     1. Age group between 20-30 is common.
     2. Height ranging between ((Height:180-190)) is found to be common .


# Cycling

In [17]:
Cycling = makeSelectiveSportDataset(data,'Cycling',['Sex','Age', 'Height','Weight','Team','Medal'])

Cycling= convertDataForApriori(Cycling)
Cycling.head()

freq_Itemsets_Cycling = applyApriori(Cycling,0.4)
print("Cycling Data")

print(freq_Itemsets_Cycling)

Cycling Data
     support                               itemsets  length
4   0.492478        ((Height:180-190), (Age:20-30))       2
5   0.536025          ((Weight:70-80), (Age:20-30))       2
6   0.704671                       ((Age:20-30), M)       2
7   0.471101     ((Height:180-190), (Weight:70-80))       2
8   0.601742                  ((Height:180-190), M)       2
9   0.619161                    ((Weight:70-80), M)       2
10  0.488519     ((Height:180-190), (Age:20-30), M)       3
11  0.515439       ((Weight:70-80), (Age:20-30), M)       3
12  0.468725  ((Height:180-190), (Weight:70-80), M)       3


### Result:
     1. Age group between 20-30 is common.
     2. Age-Height-Weight ranging between (Age:20-30),(Height:180-190), (Weight:70-80)) is found to be doing well .
    

# Diving 

In [18]:
Diving = makeSelectiveSportDataset(data,'Diving',['Sex','Age', 'Height','Weight','Team','Medal'])
Diving= convertDataForApriori(Diving)
Diving.head()

freq_Itemsets_Diving = applyApriori(Diving,0.4)
print("Diving Data")

freq_Itemsets_Diving

Diving Data


,support,itemsets,length
3,0.40281,"((Age:20-30), M)",2


### Result:
     1. Age group between 20-30 is common.
     2. Height and Weight does not matter.


# Football

In [19]:
Football = makeSelectiveSportDataset(data,'Football',['Sex','Age', 'Height','Weight','Team','Medal'])
Football= convertDataForApriori(Football)
Football.head()

freq_Itemsets_Football = applyApriori(Football,0.4)
print("Footbal Data")
print(freq_Itemsets_Football)

Footbal Data
     support                               itemsets  length
4   0.534691          ((Weight:70-80), (Age:20-30))       2
5   0.682368                       ((Age:20-30), M)       2
6   0.417568     ((Height:180-190), (Weight:70-80))       2
7   0.473584                  ((Height:180-190), M)       2
8   0.616805                    ((Weight:70-80), M)       2
9   0.518778       ((Weight:70-80), (Age:20-30), M)       3
10  0.414386  ((Height:180-190), (Weight:70-80), M)       3


### Result:
     1. Age group between 20-30 has a high support, hence greater chance of winning a medal.
     2. Age-Height-Weight ranging between (Age:20-30),(Height:180-190), (Weight:70-80)) is found to be doing well .
    

# Gymnastics

In [20]:
Gymnastics = makeSelectiveSportDataset(data,'Gymnastics',['Sex','Age', 'Height','Weight','Team','Medal'])

Gymnastics= convertDataForApriori(Gymnastics)

freq_Itemsets_Gymnastics = applyApriori(Gymnastics,0.4)
print("Gymnastics Data")

print(freq_Itemsets_Gymnastics)

Gymnastics Data
    support                            itemsets  length
4  0.545656                    ((Age:20-30), M)       2
5  0.466755  ((Weight:70-80), (Height:180-190))       2
6  0.436170                 ((Weight:70-80), M)       2


### Result:
     1. Age group between 20-30 is common.
     2. Height and Weight ((Height:180-190), (Weight:70-80)) is found to be common .


# Handball

In [21]:
Handball = makeSelectiveSportDataset(data,'Handball',['Sex','Age', 'Height','Weight','Team','Medal'])
Handball= convertDataForApriori(Handball)

freq_Itemsets_Handball = applyApriori(Handball,0.4)
print("HandBall Data")

print(freq_Itemsets_Handball)

HandBall Data
    support          itemsets  length
3  0.406604  ((Age:20-30), M)       2


### Result:
     1. Age group between 20-30 is common.
     2. Height and Weight does not matter.


# Judo

In [22]:
Judo = makeSelectiveSportDataset(data,'Judo',['Sex','Age', 'Height','Weight','Team','Medal'])
Judo= convertDataForApriori(Judo)

freq_Itemsets_Judo = applyApriori(Judo,0.4)
print("Judo Data")
print(freq_Itemsets_Judo)

Judo Data
    support               itemsets  length
3  0.425960  (Bronze, (Age:20-30))       2
4  0.563071       ((Age:20-30), M)       2


### Result:
     1. Age group between 20-30 is common and have won many bronze medals
     2. Height and Weight does not matter.


# Speed Skating

In [23]:
SpeedSkt = makeSelectiveSportDataset(data,'Speed Skating',['Sex','Age', 'Height','Weight','Team','Medal'])
SpeedSkt= convertDataForApriori(SpeedSkt)
SpeedSkt.head()

freq_Itemsets_SpeedSkt = applyApriori(SpeedSkt,0.4)
print("Speed Skating Data")

print(freq_Itemsets_SpeedSkt)

Speed Skating Data
    support               itemsets  length
5  0.486207       ((Age:20-30), M)       2
6  0.403448  ((Height:180-190), M)       2


### Result:
     1. Age group between 20-30 is common.
     2. Height ranging between ((Height:180-190)) is common	


# $$ $$

# Conclusion

## I have applied the Apriori algorithm on a few selective sports  to  find out what are common traits (Winning DNA) among the medal winnners.
### The sports list:
    1. Athletics
    2. Archery
    3. Basket ball
    4. Boxing
    5. Cycling
    6. Diving
    7. Football
    8. Gymnastics
    9. Handball
    10. Judo
    11. Speed Skating
    
   ### In general, Age ranging between (20-30),the height ranging between (180-190)cms and the weight ranging between (70-80)kg was found common among most of these sports. The respective results have been written below the code.

In [24]:
'''
med = ['Gold','Silver','Bronze']
medalC1 = data[data.Medal.isin(med)]
medalC = medalC1.groupby(['Team']).count()
medalC.reset_index(drop = False,inplace = True)
medalC = medalC[['Team','Medal']]

#medalC = medalC.reset_index(drop=False)
medalCount = medalC[['Team','Medal']]
medalCount.sort_values('Team',inplace = True)
medalCount.head()
'''

"\nmed = ['Gold','Silver','Bronze']\nmedalC1 = data[data.Medal.isin(med)]\nmedalC = medalC1.groupby(['Team']).count()\nmedalC.reset_index(drop = False,inplace = True)\nmedalC = medalC[['Team','Medal']]\n\n#medalC = medalC.reset_index(drop=False)\nmedalCount = medalC[['Team','Medal']]\nmedalCount.sort_values('Team',inplace = True)\nmedalCount.head()\n"